In [ ]:
!pip install opencv-contrib-python-headless
!pip install sk-video
!pip install pytube
!pip install humanize

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import scipy
import pandas as pd

import cv2
import skvideo.io
import imageio
import pytube

import os
import time
import csv
import re
import humanize
from tqdm import tqdm

from skimage.measure import compare_mse, compare_ssim
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import hamming

In [3]:
save_path = '/home/idies/workspace/Storage/Cong/persistent/video/data'
data_path = '/home/idies/workspace/Storage/Cong/persistent/video/data'
result_path = '/home/idies/workspace/Storage/Cong/persistent/video/result'

In [4]:
def image_crop(h1, h2, l1, l2, original):
    return original[h1:h2, l1:l2, :]

In [5]:
def get_FPS(video_loc):
    video = cv2.VideoCapture(video_loc)
    fps = video.get(cv2.CAP_PROP_FPS)
    return fps

In [6]:
def get_histogram(img):
    h, w = img.shape
    hist = [0.0] * 256
    for i in range(h):
        for j in range(w):
            hist[img[i, j]] += 1
    return np.array(hist) / (h * w)

In [7]:
def normalize_exposure(img):
    '''
    Normalize the exposure of an image.
    '''
    img = img.astype(int)
    hist = get_histogram(img)
    # get the sum of vals accumulated by each position in hist
    cdf = np.array([sum(hist[:i+1]) for i in range(len(hist))])
    # determine the normalization values for each unit of the cdf
    sk = np.uint8(255 * cdf)
    # normalize each position in the output image
    h, w = img.shape
    normalized = np.zeros_like(img)
    for i in range(0, h):
        for j in range(0, w):
            normalized[i, j] = sk[img[i, j]]
    return normalized.astype(int)

In [8]:
def compare_emd(img1, img2):
    img1 = normalize_exposure(img1)
    img2 = normalize_exposure(img2)
    return wasserstein_distance(get_histogram(img1), get_histogram(img2))

In [9]:
def compare_orb(img1, img2, threshold):
    orb = cv2.ORB_create()
    kp_1, desc_1 = orb.detectAndCompute(img1, None)
    kp_2, desc_2 = orb.detectAndCompute(img2, None)
    if desc_1 is None or desc_2 is None:
        return 0
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(desc_1, desc_2)
    if len(matches) == 0:
        return 0
    similar = [i for i in matches if i.distance < threshold]
    return len(similar) / len(matches)

In [10]:
def dhash(img, hashSize=8):
    # resize the input image, adding a single column (row) so we can compute the gradient
    resized_col = cv2.resize(img, (hashSize+1,hashSize))
    resized_row = cv2.resize(img, (hashSize,hashSize+1))
 
    # compute the (relative) gradient between adjacent pixels
    diff_col = resized_col[:,1:] > resized_col[:,:-1]
    diff_row = resized_row[1:,:] > resized_row[:-1,:]

    # convert the difference image to a hash
    return sum([2**i for (i, v) in enumerate(np.append(diff_col.flatten(), diff_row.flatten())) if v])

In [11]:
def compare_dhash(img1, img2, hashSize=8):
    h1 = [int(d) for d in str(int(dhash(img1)))]
    h2 = [int(d) for d in str(int(dhash(img2)))]
    
    if len(h1) == len(h2):
        dHash = hamming(h1, h2)
    else:
        dHash = 1
    return dHash

In [12]:
def compare_phash(img1, img2):
    phash = cv2.img_hash_PHash.create()
    return phash.compare(phash.compute(img1), phash.compute(img2))

In [13]:
def get_similar_image(img, image_df, method, ascending, start, end, h1, h2, l1, l2):
    results_dict = {}
    img = image_crop(h1, h2, l1, l2, img)
    if method == 'EMD' or method == 'ORB' or method == 'dHash':
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
    for i in range(start, min(end, image_df.shape[0])):
        #i = i - image_df['img_index'][0]
        temp_img = imread(image_df['img_path'][i])
        temp_img = image_crop(h1, h2, l1, l2, temp_img)
        if method == 'MSE':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_mse(img, temp_img)]
        elif method == 'EMD':
            temp_img = cv2.cvtColor(temp_img, cv2.COLOR_RGB2GRAY)
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_emd(img, temp_img)]
        elif method == 'ORB':
            temp_img = cv2.cvtColor(temp_img, cv2.COLOR_RGB2GRAY)
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_orb(img, temp_img, threshold)]
        elif method == 'dHash':
            temp_img = cv2.cvtColor(temp_img, cv2.COLOR_RGB2GRAY)
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_dhash(img, temp_img)]
        elif method == 'pHash':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_phash(img, temp_img)]
        else:
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_ssim(img, temp_img, multichannel=True)]
    
    results_df = pd.DataFrame.from_dict(results_dict, orient='index')
    results_df.columns = ['img_path', 'img_index', method]
    results_df = results_df.sort_values(by=method, ascending=ascending)
    return results_df

In [14]:
def weighted_index(df, n, method, ascending):
    temp = df.copy()[:n]
    if ascending:
        temp['weight'] = (1/temp[method]) / sum(1/temp[method])
    else:
        temp['weight'] = temp[method] / sum(temp[method])
    return sum(temp['img_index']*temp['weight'])

In [15]:
prior_time = 60
threshold = 70

In [126]:
def lap_time2(initial_idx, end, fps, topn, df, method, ascending, h1, h2, l1, l2):
    results_dict = {}
    start = initial_idx + np.round(fps*prior_time).astype(int)
    
    img_idxs = np.arange(initial_idx, initial_idx+5).tolist()
    #img_idxs = np.arange(initial_idx+5, initial_idx+10).tolist()
    #img_idxs = np.arange(initial_idx+10, initial_idx+15).tolist()
    #img_idxs = np.arange(initial_idx+15, initial_idx+20).tolist()
    #img_idxs = np.arange(initial_idx+20, initial_idx+25).tolist()
    #img_idxs = np.arange(initial_idx+25, np.round(initial_idx+fps).astype(int)).tolist()
    
    #img_idxs = np.arange(initial_idx, initial_idx+10).tolist()
    #img_idxs = np.arange(initial_idx+10, initial_idx+20).tolist()
    #img_idxs = np.arange(initial_idx+20, np.round(initial_idx+fps).astype(int)).tolist()
    
    #img_idxs = np.choice(np.arange(initial_idx, np.round(initial_idx+fps).astype(int)), 10).tolist()
    
    for idx in img_idxs:
        img = imread(df['img_path'][idx])
        start_time = time.time()
        similar_imgs = get_similar_image(img, df, method, ascending, start, end, h1, h2, l1, l2)
        elapsedTime = time.time() - start_time
        results_dict[idx] = [df['img_path'][idx], df['img_index'][idx], (similar_imgs['img_index'].iloc[0]-df['img_index'][idx])/fps, (weighted_index(similar_imgs, topn, method, ascending)-df['img_index'][idx])/fps, elapsedTime, method]
    
    results_df = pd.DataFrame.from_dict(results_dict, orient='index')
    results_df.columns = ['img_path', 'img_index', 'lap_time', 'weighted_lap_time', 'elapsedTime', 'method']
    return results_df

In [127]:
df = pd.read_csv(os.path.join(data_path, 'train153240_raw.csv'))

initial_idx = 26982
end = 31455

fps = 29.93943500598125
topn = 5

method = 'MSE'
ascending = True

h1 = 400
h2 = 810
l1 = 250
l2 = 1920

In [128]:
test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, h1, h2, l1, l2)
test['lap'] = 11
test.to_csv(os.path.join(result_path, '153240', 'lap11_MSE_6.csv'), index=False)

In [157]:
temp1 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_1.csv'))
temp2 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_2.csv'))
temp3 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_3.csv'))
temp4 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_4.csv'))
temp5 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_5.csv'))
temp6 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_6.csv'))
temp = temp1.append(temp2)
temp = temp.append(temp3)
temp = temp.append(temp4)
temp = temp.append(temp5)
temp = temp.append(temp6)


# initial_idx
print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# start
print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# end
print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

26916
28712
31335


In [158]:
temp.to_csv(os.path.join(result_path, '153240', 'lap11_MSE.csv'), index=False)

In [159]:
temp = pd.read_csv('./result/153240/lap11_MSE.csv')
np.mean(temp['lap_time'])

87.6168838682474

In [153]:
# initial_idx = 0
# end = 3600

# initial_idx = 2778
# end = 7353

# initial_idx = 4650
# end = 8303

# initial_idx = 8220
# end = 12915

# initial_idx = 10926
# end = 15425

# initial_idx = 13651
# end = 18176

# initial_idx = 16332
# end = 20817

# initial_idx = 19010
# end = 23489

# initial_idx = 21669
# end = 26120

# initial_idx = 24293
# end = 28741

In [57]:
# df = pd.read_csv(os.path.join(data_path, 'train091108_raw.csv'))

# initial_idx = 21682
# end = 26174

# fps = 29.93930396046777
# topn = 5

# method = 'MSE'
# ascending = True

# h1 = 400
# h2 = 760
# l1 = 250
# l2 = 1920

In [58]:
# test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, h1, h2, l1, l2)
# test['lap'] = 8
# test.to_csv(os.path.join(result_path, '091108', 'lap8_MSE_3.csv'), index=False)

In [60]:
# temp1 = pd.read_csv(os.path.join(result_path, '091108', 'lap8_MSE_1.csv'))
# temp2 = pd.read_csv(os.path.join(result_path, '091108', 'lap8_MSE_2.csv'))
# temp3 = pd.read_csv(os.path.join(result_path, '091108', 'lap8_MSE_3.csv'))
# temp = temp1.append(temp2)
# temp = temp.append(temp3)

# # initial_idx
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# # start
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# # end
# print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

21682
23478
26174


In [61]:
# temp.to_csv(os.path.join(result_path, '091108', 'lap8_MSE.csv'), index=False)

In [17]:
# temp = pd.read_csv('./result/091108/lap9_MSE.csv')
# np.mean(temp['lap_time'])

89.55006670184385

In [59]:
# initial_idx = 0
# end = 3600

# initial_idx = 2766
# end = 7323

# initial_idx = 5527
# end = 10083

# initial_idx = 8249
# end = 12763

# initial_idx = 10942
# end = 15434

# initial_idx = 13629
# end = 18110

# initial_idx = 16316
# end = 20801

# initial_idx = 18984
# end = 23448